In [1]:
from linkedin_api import Linkedin


# GET a profile
profile = api.get_profile('billy-g')

In [133]:
profile

In [31]:
from urllib.parse import urlencode


default_params = {
    "decorationId": "com.linkedin.voyager.dash.deco.jobs.search.JobSearchCardsCollection-174",
    "count": 0,
    "q": "jobSearch",
    "query": "",
    "start": len([]) + 0,
    "filters": "List((key:savedPostType,value:List(ALL)))"
}

encoded_params = urlencode(default_params, safe='(),:')
encoded_params

'decorationId=com.linkedin.voyager.dash.deco.jobs.search.JobSearchCardsCollection-174&count=0&q=jobSearch&query=&start=0&filters=List((key:savedPostType,value:List(ALL)))'

In [32]:
query = f"""
/graphql
?variables=(
    start:{default_params['start']},
    query:(
        flagshipSearchIntent:SEARCH_MY_ITEMS_SAVED_POSTS,
        queryParameters:{default_params['filters']}
    )
)
&queryId=voyagerSearchDashClusters.a2b606e8c1f58b3cf72fb5d54a2a57e7
""".replace("\n","").replace("  ","")
print(query)

/graphql?variables=(start:0,query:(flagshipSearchIntent:SEARCH_MY_ITEMS_SAVED_POSTS,queryParameters:List((key:savedPostType,value:List(ALL)))))&queryId=voyagerSearchDashClusters.a2b606e8c1f58b3cf72fb5d54a2a57e7


In [33]:
res = api._fetch(query)

In [34]:
post_list = res.json()["data"]["searchDashClustersByAll"]["elements"][0]["items"]
len(post_list)

10

In [45]:
res.json()["data"]["searchDashClustersByAll"]["metadata"]["paginationToken"]

'dXJuOmxpOmFjdGl2aXR5OjcyMzI0Njg1MzQyNDM1Njk2NjQtMTcyNDgzODUxNDI2Mg=='

In [71]:
extracted_posts = [
    {
        "author": post["item"]["entityResult"]["title"]["text"],
        "author_catchline": post["item"]["entityResult"]["primarySubtitle"]["text"],
        "post_summary": post["item"]["entityResult"]["summary"]["text"],
        "post_url": post["item"]["entityResult"]["navigationUrl"],
    }
    for post in post_list
]
extracted_posts

[{'author': 'Marie Stephen Leo',
  'author_catchline': 'I help Data enthusiasts learn about AI, DS, DE, Analytics Engineering, and MLOps | Linkedin Top Voice',
  'post_summary': 'LLM Structured Output Benchmarks: Find the best LLM structured data parsing framework for your task!\n\nOver the past few weeks, I’ve continued to update my benchmarking library, which just crossed 100 stars on GitHub!\n\n🏆 Added synthetic data generation task. Benchmark results are live for most of the frameworks.\n\n🐸 Updated Mirascope to V1, which significantly simplified the code required by the framework. I really like the simplicity of the V1 Mirascope API. https://lnkd.in/gNj3TJCu\n\n🔆 Added a vanilla OpenAI using the Structured Outputs capability. It performs really well and is probably all you need for most tasks except synthetic data generation. I need to dig into the actual outputs soon to understand its weaknesses. https://lnkd.in/g-eWruza\n\nMeanwhile, I’ve been sourcing a suitable classification 

In [30]:
res = api._fetch(
    #f"includeWebMetadata=true"
    f"/graphql?variables=(start:{default_params['start']},"
    f"query:("
    "count:1,"
    f"flagshipSearchIntent:SEARCH_MY_ITEMS_SAVED_POSTS,"
    f"queryParameters:{default_params['filters']}))&queryId=voyagerSearchDashClusters"
    f".a2b606e8c1f58b3cf72fb5d54a2a57e7"
)

NameError: name 'default_params' is not defined

In [144]:
import time
import random


def get_bookmarked_posts(limit=-1):
    results = []
    pagination_token=None
    start=None
    count=10

    map_date = {
       "m": "minutes",
       "h": "hours",
       "d": "days",
       "w": "weeks",
       "mo": "months",
    }
    try:
        while len(results) < limit or limit == -1:
            if limit > -1 and limit - len(results) < count:
                count = limit - len(results)

            query = f"""
            /graphql
            ?variables=(
                {f"{start}," if start else ""}
                count:{count},
                {f"{pagination_token}," if pagination_token else ""}
                query:(
                    flagshipSearchIntent:SEARCH_MY_ITEMS_SAVED_POSTS,
                    queryParameters:List((key:savedPostType,value:List(ALL)))
                )
            )
            &queryId=voyagerSearchDashClusters.a2b606e8c1f58b3cf72fb5d54a2a57e7
            """.replace("\n","").replace("  ","")   
            print(query)

            res = api._fetch(query)

            post_list = res.json()["data"]["searchDashClustersByAll"]["elements"][0]["items"]

            if len(post_list) == 0:
                break

            for post in post_list:
                post_date = post["item"]["entityResult"]["secondarySubtitle"]["text"]
                try:
                    post_date_extract = post_date.split("•")[-2].strip()
                    post_formated_date = datetime.datetime.now() - datetime.timedelta(**{map_date[post_date_extract[-1]]:int(post_date_extract[:-1])})
                except KeyError:
                    post_date_extract = None
                    post_formated_date = None

                extracted_posts = {
                        "author": post["item"]["entityResult"]["title"]["text"],
                        "author_catchline": post["item"]["entityResult"]["primarySubtitle"]["text"],
                        "author_member_id": post["item"]["entityResult"]["actorTrackingUrn"].split(":")[-1],
                        "post_summary": post["item"]["entityResult"]["summary"]["text"],
                        "post_url": post["item"]["entityResult"]["navigationUrl"],
                        "post_activity_id": post["item"]["entityResult"]["trackingUrn"].split(":")[-1],
                        "post_date_lkd": post_date,
                        "post_date_extract": post_date_extract,
                        "post_date_formatted": post_formated_date,
                    }

                results.append(extracted_posts)

            pagination_token = f'paginationToken:{res.json()["data"]["searchDashClustersByAll"]["metadata"]["paginationToken"]}'
            start = f"start:{len(results)}"

            print(f"Total {len(results)} values retrieved")

            time.sleep(random.randint(1,3))
    except:
        return results

    return results

In [137]:
post_date = post_list[0]["item"]["entityResult"]["secondarySubtitle"]["text"].split("•")[-2].strip()
post_date

'9h'

In [141]:
post_date = "1yr • Edited •".split("•")[-2].strip()
post_date

'Edited'

In [132]:
import datetime

datetime.datetime.now() - datetime.timedelta(**{map_date[post_date[-1]]:int(post_date[:-1])})

datetime.datetime(2024, 8, 30, 2, 56, 0, 620477)

In [95]:

map_date["w"]

'weeks'

In [145]:
of_10 = get_bookmarked_posts(limit=800)

/graphql?variables=(count:10,query:(flagshipSearchIntent:SEARCH_MY_ITEMS_SAVED_POSTS,queryParameters:List((key:savedPostType,value:List(ALL)))))&queryId=voyagerSearchDashClusters.a2b606e8c1f58b3cf72fb5d54a2a57e7
Total 10 values retrieved
/graphql?variables=(start:10,count:10,paginationToken:dXJuOmxpOmFjdGl2aXR5OjcyMzI0Njg1MzQyNDM1Njk2NjQtMTcyNDgzODUxNDI2Mg==,query:(flagshipSearchIntent:SEARCH_MY_ITEMS_SAVED_POSTS,queryParameters:List((key:savedPostType,value:List(ALL)))))&queryId=voyagerSearchDashClusters.a2b606e8c1f58b3cf72fb5d54a2a57e7
Total 20 values retrieved
/graphql?variables=(start:20,count:10,paginationToken:dXJuOmxpOmFjdGl2aXR5OjcyMzM3MzIwNTA3NDEwNjM2ODEtMTcyNDY2MzY0ODEyOA==,query:(flagshipSearchIntent:SEARCH_MY_ITEMS_SAVED_POSTS,queryParameters:List((key:savedPostType,value:List(ALL)))))&queryId=voyagerSearchDashClusters.a2b606e8c1f58b3cf72fb5d54a2a57e7
Total 30 values retrieved
/graphql?variables=(start:30,count:10,paginationToken:dXJuOmxpOmFjdGl2aXR5OjcyMzMxMzE4NzU3MTA0MzEy

In [146]:
import pandas as pd

df_bookmarks = pd.DataFrame(of_10)
df_bookmarks
#print(df_bookmarks["post_summary"][0])

,author,author_catchline,author_member_id,post_summary,post_url,post_activity_id,post_date_lkd,post_date_extract,post_date_formatted
0,Marie Stephen Leo,"I help Data enthusiasts learn about AI, DS, DE...",69170718,LLM Structured Output Benchmarks: Find the bes...,https://www.linkedin.com/feed/update/urn:li:ac...,7235088849331183616,11h •,11h,2024-08-30 09:24:53.447237
1,Ansgar Wolsing,Director Data & Analytics @ Kaiser X Labs | Da...,602321657,Did you know you can export a ggplot2 chart as...,https://www.linkedin.com/feed/update/urn:li:ac...,7234956777983782912,20h •,20h,2024-08-30 00:24:53.447268
2,Sarthak Rastogi,AI engineer building agents | Experienced in L...,715062628,Apple’s new Superposition Prompting method imp...,https://www.linkedin.com/feed/update/urn:li:ac...,7234892590578311168,1d •,1d,2024-08-29 20:24:53.447276
3,Daniel Lee,Data/AI Jobs with Datainterview.com 🚀 | AI Con...,447871011,How do you become a 𝗙𝘂𝗹𝗹𝘀𝘁𝗮𝗰𝗸 𝗗𝗮𝘁𝗮 𝗦𝗰𝗶𝗲𝗻𝘁𝗶𝘀𝘁 o...,https://www.linkedin.com/feed/update/urn:li:ac...,7233488313716805632,4d •,4d,2024-08-26 20:24:53.447281
4,Dat Tran,"VP of AI/ML at Beams Safety AI 🤖 | Advisor, In...",71711730,One of my favorite integration for tqdm - the ...,https://www.linkedin.com/feed/update/urn:li:ac...,7234809803888320512,1d •,1d,2024-08-29 20:24:53.447286
...,...,...,...,...,...,...,...,...,...
795,Marco Gorelli,"pandas, Polars, Narwhals | Senior Software Eng...",825876736,🛑 Don't collect-block yourself!\n\n⚡ The Polar...,https://www.linkedin.com/feed/update/urn:li:ac...,7111725378674507776,1yr •,None,NaT
796,Harry Powell,Data science leader with track record of innov...,167072583,Here's an explainer video about estimating dem...,https://www.linkedin.com/feed/update/urn:li:ac...,7110193713418514432,1yr •,None,NaT
797,Liam Brannigan,Polars Training | O'Reilly,603401624,Streaming mode in Polars lets you work with la...,https://www.linkedin.com/feed/update/urn:li:ac...,7109839031617630209,1yr •,None,NaT
798,Marco Gorelli,"pandas, Polars, Narwhals | Senior Software Eng...",825876736,🏮 Utilities for Polars ternary expressions hav...,https://www.linkedin.com/feed/update/urn:li:ac...,7109847293142396929,1yr •,None,NaT


In [148]:
df_bookmarks.to_csv("df_bookmarks_800.csv", index=False)

In [80]:
post_list[0]["item"]["entityResult"]["trackingUrn"].split(":")[-1]

'urn:li:activity:7235088849331183616'

In [82]:
post_list[0]["item"]["entityResult"]["actorTrackingUrn"]

'urn:li:member:69170718'